<a href="https://colab.research.google.com/github/karta2552442a/jacky/blob/master/Pokemon_Image_Search.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!mkdir dataset

用mkdir指令建立圖像資料夾dataset

In [0]:
!mkdir dataset/Gengar
!mkdir dataset/Charizard
!mkdir dataset/Dragonite
!mkdir dataset/Gyarados
!mkdir dataset/Snorlax

#dataset/Gengar
#dataset/Charizard
#dataset/Dragonite
#dataset/Gyarados
#dataset/Snorlax

再用mkdir指令在建立好的dataset資料夾中新增五種寶可夢的資料夾，用來保存圖片

In [0]:
! pip install requests
from requests import exceptions
import requests
import cv2
import os
import gevent

# poke name to download
pokemon = 'Gengar'
output = 'dataset/Gengar'

API_KEY = "b8c532437a5d42efb755e2e9d840df1e"
MAX_RESULTS = 250
GROUP_SIZE = 50

# set the endpoint API URL
URL = "https://api.cognitive.microsoft.com/bing/v7.0/images/search"

# when attempting to download images from the web both the Python
# programming language and the requests library have a number of
# exceptions that can be thrown so let's build a list of them now
# so we can filter on them
EXCEPTIONS = {IOError, FileNotFoundError, exceptions.RequestException, exceptions.HTTPError, exceptions.ConnectionError,
              exceptions.Timeout}

# store the search term in a convenience variable then set the
# headers and search parameters
term = pokemon
headers = {"Ocp-Apim-Subscription-Key": API_KEY}
params = {"q": term, "offset": 0, "count": GROUP_SIZE}

# make the search
print("[INFO] searching Bing API for '{}'".format(term))
search = requests.get(URL, headers=headers, params=params)
search.raise_for_status()

# grab the results from the search, including the total number of
# estimated results returned by the Bing API
results = search.json()
estNumResults = min(results["totalEstimatedMatches"], MAX_RESULTS)
print("[INFO] {} total results for '{}'".format(estNumResults, term))

# initialize the total number of images downloaded thus far
total = 0


def grab_page(url, ext, total):

    try:
        # total += 1
        print("[INFO] fetching: {}".format(url))
        r = requests.get(url, timeout=30)
        # build the path to the output image

        #here total is only for filename creation
        p = os.path.sep.join([output, "{}{}".format(
            str(total).zfill(8), ext)])

        # write the image to disk
        f = open(p, "wb")
        f.write(r.content)
        f.close()

        # try to load the image from disk
        image = cv2.imread(p)

        # if the image is `None` then we could not properly load the
        # image from disk (so it should be ignored)
        if image is None:
            print("[INFO] deleting: {}".format(p))
            os.remove(p)
            return

    # catch any errors that would not unable us to download the
    # image
    except Exception as e:
        # check to see if our exception is in our list of
        # exceptions to check for
        if type(e) in EXCEPTIONS:
            print("[INFO] skipping: {}".format(url))
            return 

# loop over the estimated number of results in `GROUP_SIZE` groups
for offset in range(0, estNumResults, GROUP_SIZE):
    # update the search parameters using the current offset, then
    # make the request to fetch the results
    print("[INFO] making request for group {}-{} of {}...".format(
        offset, offset + GROUP_SIZE, estNumResults))
    params["offset"] = offset
    search = requests.get(URL, headers=headers, params=params)
    search.raise_for_status()
    results = search.json()
    print("[INFO] saving images for group {}-{} of {}...".format(
        offset, offset + GROUP_SIZE, estNumResults))
    # loop over the results
    jobs = []
    for v in results["value"]:
        total += 1
        ext = v["contentUrl"][v["contentUrl"].rfind("."):]
        url = v["contentUrl"]
        
        # create gevent job
        jobs.append(gevent.spawn(grab_page, url, ext, total))

    # wait for all jobs to complete
    gevent.joinall(jobs, timeout=10)
    print(total)

[INFO] searching Bing API for 'Gengar'


HTTPError: ignored

利用Microsoft’s Bing Image Search的功能搜尋圖片，先在Microsoft的Cognitive Services申請Bing Search專用的API KEY，pokemon用寶可夢的名字作為搜尋的關鍵字，搜尋到的圖片若可以存取便儲存在剛剛建好的資料夾，以output定義存入圖片的路徑，GROUP_SIZE=50代表抓圖時以50張為一組，設定MAX_RESULTS=250，總共250張圖片作為樣本

In [0]:
!git clone https://github.com/jrosebr1/imutils.git

Cloning into 'imutils'...
remote: Enumerating objects: 660, done.
remote: Total 660 (delta 0), reused 0 (delta 0), pack-reused 660
Receiving objects: 100% (660/660), 6.88 MiB | 13.46 MiB/s, done.
Resolving deltas: 100% (383/383), done.


clone imutils工具到程式中，imutils為圖像處理的工具

In [0]:
# import the necessary packages
from keras.models import Sequential
from keras.layers.normalization import BatchNormalization
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import MaxPooling2D
from keras.layers.core import Activation
from keras.layers.core import Flatten
from keras.layers.core import Dropout
from keras.layers.core import Dense
from keras import backend as K

class SmallerVGGNet:
	@staticmethod
	def build(width, height, depth, classes):
		# initialize the model along with the input shape to be
		# "channels last" and the channels dimension itself
		model = Sequential()
		inputShape = (height, width, depth)
		chanDim = -1
 
		# if we are using "channels first", update the input shape
		# and channels dimension
		if K.image_data_format() == "channels_first":
			inputShape = (depth, height, width)
			chanDim = 1
      
      # CONV => RELU => POOL
		model.add(Conv2D(32, (3, 3), padding="same",
			input_shape=inputShape))
		model.add(Activation("relu"))
		model.add(BatchNormalization(axis=chanDim))
		model.add(MaxPooling2D(pool_size=(3, 3)))
		model.add(Dropout(0.25))
    
    # (CONV => RELU) * 2 => POOL
		model.add(Conv2D(64, (3, 3), padding="same"))
		model.add(Activation("relu"))
		model.add(BatchNormalization(axis=chanDim))
		model.add(Conv2D(64, (3, 3), padding="same"))
		model.add(Activation("relu"))
		model.add(BatchNormalization(axis=chanDim))
		model.add(MaxPooling2D(pool_size=(2, 2)))
		model.add(Dropout(0.25))
    
    # (CONV => RELU) * 2 => POOL
		model.add(Conv2D(128, (3, 3), padding="same"))
		model.add(Activation("relu"))
		model.add(BatchNormalization(axis=chanDim))
		model.add(Conv2D(128, (3, 3), padding="same"))
		model.add(Activation("relu"))
		model.add(BatchNormalization(axis=chanDim))
		model.add(MaxPooling2D(pool_size=(2, 2)))
		model.add(Dropout(0.25))
    
    # first (and only) set of FC => RELU layers
		model.add(Flatten())
		model.add(Dense(1024))
		model.add(Activation("relu"))
		model.add(BatchNormalization())
		model.add(Dropout(0.5))
 
		# softmax classifier
		model.add(Dense(classes))
		model.add(Activation("softmax"))
 
		# return the constructed network architecture
		return model
  
  
  

Using TensorFlow backend.


建立SmallerVGGNet模組，引入keras的模組，定義四個參數width, height, depth, classes，分別是圖像寬度，圖像高度，圖像的深度及數據集中的類數，即寶可夢的種類數，接著加入layer到模組中，首先是CONV => RELU => POOL，卷積層有32個filters和3X3的kernel，POOL層使用 3 x 3大小將空間尺寸從96 x 96 快速縮小到32 x 32，並用dropout25%來隨機斷開目前的layer防止過度擬合，接著是（CONV = > RELU ）* 2，多個CONV和 RELU層堆疊 在一起 ，將filter大小從32增加到64，並把pool size從3X3減少到2X2，接著再加入一組（CONV = > RELU ）* 2 = > POOL，filter大小增加到128，Dense(1024)用來完全連接指定的layer，並把數據線性及歸一化，在fully-connected layers執行最後的dropout，大小為50%

In [0]:
! pip install --upgrade imutils

Requirement already up-to-date: imutils in /usr/local/lib/python3.6/dist-packages (0.5.2)


更新imutils

In [0]:
# set the matplotlib backend so figures can be saved in the background
import matplotlib
matplotlib.use("Agg")

# import the necessary packages
from keras.preprocessing.image import ImageDataGenerator
from keras.optimizers import Adam
from keras.preprocessing.image import img_to_array
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
#from smallervggnet import SmallerVGGNet
import matplotlib.pyplot as plt
from imutils import paths
import numpy as np
import argparse
import random
import pickle
import cv2
import os

# construct the argument parse and parse the arguments
dataset= 'dataset'
modelz='pokedex.model'
labelbin='lb.pickle'
plot='plot.png'


#args = easydict.EasyDict({
        #"dataset":True,
        #"model":True,
        #"labelbin":True,
      #  "plot":"plot.png",
        
#})

 #construct the argument parse and parse the arguments
"""ap = argparse.ArgumentParser()
ap.add_argument("-d", "--dataset", required=True,
	help="path to input dataset (i.e., directory of images)")
ap.add_argument("-m", "--model", required=True,
  help="path to output model")
ap.add_argument("-l", "--labelbin", required=True,
	help="path to output label binarizer")
ap.add_argument("-p", "--plot", type=str, default="plot.png",
	help="path to output accuracy/loss plot")
args = vars(ap.parse_args())"""

# initialize the number of epochs to train for, initial learning rate,
# batch size, and image dimensions
EPOCHS = 100
INIT_LR = 1e-3
BS = 32
IMAGE_DIMS = (96, 96, 3)

# initialize the data and labels
data = []
labels = []


# grab the image paths and randomly shuffle them
print("[INFO] loading images...")
#imagePaths = sorted(list(paths.list_images(args["dataset"])))
imagePaths = sorted(list(paths.list_images(dataset)))
random.seed(42)
random.shuffle(imagePaths)

# loop over the input images
for imagePath in imagePaths:
	# load the image, pre-process it, and store it in the data list
	image = cv2.imread(imagePath)
	image = cv2.resize(image, (IMAGE_DIMS[1], IMAGE_DIMS[0]))
	image = img_to_array(image)
	data.append(image)
 
	# extract the class label from the image path and update the
	# labels list
	label = imagePath.split(os.path.sep)[-2]
	labels.append(label)

# scale the raw pixel intensities to the range [0, 1]
data = np.array(data, dtype="float") / 255.0
labels = np.array(labels)
print("[INFO] data matrix: {:.2f}MB".format(
	data.nbytes / (1024 * 1000.0)))

# binarize the labels
lb = LabelBinarizer()
labels = lb.fit_transform(labels)

# partition the data into training and testing splits using 80% of
# the data for training and the remaining 20% for testing
(trainX, testX, trainY, testY) = train_test_split(data,
	labels, test_size=0.2, random_state=42)

# construct the image generator for data augmentation
aug = ImageDataGenerator(rotation_range=25, width_shift_range=0.1,
	height_shift_range=0.1, shear_range=0.2, zoom_range=0.2,
	horizontal_flip=True, fill_mode="nearest")

# initialize the model
print("[INFO] compiling model...")
model = SmallerVGGNet.build(width=IMAGE_DIMS[1], height=IMAGE_DIMS[0],
	depth=IMAGE_DIMS[2], classes=len(lb.classes_))
opt = Adam(lr=INIT_LR, decay=INIT_LR / EPOCHS)
model.compile(loss="categorical_crossentropy", optimizer=opt,
	metrics=["accuracy"])

# train the network
print("[INFO] training network...")
H = model.fit_generator(
	aug.flow(trainX, trainY, batch_size=BS),
	validation_data=(testX, testY),
	steps_per_epoch=len(trainX) // BS,
	epochs=EPOCHS, verbose=1)

# save the model to disk
print("[INFO] serializing network...")
#model.save(args["model"])
model.save(modelz)

# save the label binarizer to disk
print("[INFO] serializing label binarizer...")
#f = open(args["labelbin"], "wb")
f = open(labelbin, "wb")
f.write(pickle.dumps(lb))
f.close()

# plot the training loss and accuracy
plt.style.use("ggplot")
plt.figure()
N = EPOCHS
plt.plot(np.arange(0, N), H.history["loss"], label="train_loss")
plt.plot(np.arange(0, N), H.history["val_loss"], label="val_loss")
plt.plot(np.arange(0, N), H.history["acc"], label="train_acc")
plt.plot(np.arange(0, N), H.history["val_acc"], label="val_acc")
plt.title("Training Loss and Accuracy")
plt.xlabel("Epoch #")
plt.ylabel("Loss/Accuracy")
plt.legend(loc="upper left")
#plt.savefig(args["plot"])
plt.savefig(plot)







[INFO] loading images...
[INFO] data matrix: 232.42MB
[INFO] compiling model...
Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
[INFO] training network...
Instructions for updating:
Use tf.cast instead.
Epoch 1/100
26/26 [==============================] - 54s 2s/step - loss: 2.3762 - acc: 0.3906 - val_loss: 2.7150 - val_acc: 0.4444
Epoch 2/100
26/26 [==============================] - 51s 2s/step - loss: 1.4879 - acc: 0.5244 - val_loss: 2.9183 - val_acc: 0.4213
Epoch 3/100
26/26 [==============================] - 50s 2s/step - loss: 1.3863 - acc: 0.5453 - val_loss: 1.7475 - val_acc: 0.5139
Epoch 4/100
26/26 [==============================] - 50s 2s/step - loss: 1.2099 - acc: 0.5967 - val_loss: 1.4746 - val_acc: 0.5602
Epoch 5/100
26/26 [==============================] - 50s 2s/step - loss: 1.1384 - acc: 0.6116 - val_loss: 2.9033 - val_acc: 0.4583
Epo

用此單元格進行訓練，先定義三個參數dataset,modelz,labelbin，dataset為我們存取用來訓練的圖片路徑，modelz指定訓練完模型的輸出位置，labelbin為輸出標籤位置，並定義plot，畫圖儲存訓練結果，接著設定訓練次數epochs為100，IMAGE_DIMS中設定圖像為96X96像素並加上3個channel，載入dataset的所有圖像後隨機地移動，在imagePaths迴圈裡進行調整，對dataset的每個資料夾的圖像記標籤，讓程式訓練每個資料夾中對不同寶可夢的圖像辨識，最後儲存model 和label binarizer，並繪圖plot，可以從plot.png中看出圖形的訓練過程的精準度

https://www.pyimagesearch.com/2018/04/09/how-to-quickly-build-a-deep-learning-image-dataset/

https://www.pyimagesearch.com/2018/04/16/keras-and-convolutional-neural-networks-cnns/

https://qiita.com/LittleWat/items/6e56857e1f97c842b261


In [0]:
from google.colab import files

uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))

Saving Charizard_counter.png to Charizard_counter.png
User uploaded file "Charizard_counter.png" with length 91886 bytes


上傳測試圖片

In [0]:
# import the necessary packages
from keras.preprocessing.image import img_to_array
from keras.models import load_model
import numpy as np
import argparse
import imutils
import pickle
import cv2
import os

# construct the argument parse and parse the arguments
'''ap = argparse.ArgumentParser()
ap.add_argument("-m", "--model", required=True,
	help="path to trained model model")
ap.add_argument("-l", "--labelbin", required=True,
	help="path to label binarizer")
ap.add_argument("-i", "--image", required=True,
	help="path to input image")
args = vars(ap.parse_args())'''

modelz='pokedex.model'
labelbin='lb.pickle'
image='Charizard_counter.png'

# load the image
#image = cv2.imread(args["image"])
image = cv2.imread(image)
output = image.copy()
 
# pre-process the image for classification
image = cv2.resize(image, (96, 96))
image = image.astype("float") / 255.0
image = img_to_array(image)
image = np.expand_dims(image, axis=0)

# load the trained convolutional neural network and the label
# binarizer
print("[INFO] loading network...")
#model = load_model(args["model"])
model = load_model(modelz)
#lb = pickle.loads(open(args["labelbin"], "rb").read())
lb = pickle.loads(open(labelbin, "rb").read())
 
# classify the input image
print("[INFO] classifying image...")
proba = model.predict(image)[0]
idx = np.argmax(proba)
label = lb.classes_[idx]

# we'll mark our prediction as "correct" of the input image filename
# contains the predicted label text (obviously this makes the
# assumption that you have named your testing image files this way)
#filename = args["image"][args["image"].rfind(os.path.sep) + 1:]
filename = image[image.rfind(os.path.sep) + 1:]
correct = "correct" if filename.rfind(label) != -1 else "incorrect"
 
# build the label and draw the label on the image
label = "{}: {:.2f}% ({})".format(label, proba[idx] * 100, correct)
#label = "{}: {:.2f}% ".format(label, proba[idx] * 100)
output = imutils.resize(output, width=400)
cv2.putText(output, label, (10, 25),  cv2.FONT_HERSHEY_SIMPLEX,
	0.7, (0, 255, 0), 2)
 
# show the output image
print("[INFO] {}".format(label))
cv2.imshow("Output", output)
cv2.waitKey(0)



[INFO] loading network...
[INFO] classifying image...


AttributeError: ignored